In [1]:
%matplotlib inline

import pandas as pd
import nivapy3 as nivapy
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import text
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

plt.style.use("ggplot")

# CL Vestland: Add catchment data to PostGIS

Add catchment data from the template here:

    critical_loads_2/cl_vestland/cl_vestland_stns.xlsx

In [2]:
# Connect to PostGIS
# If connecting from JupyterHub
eng = nivapy.da.connect_postgis(admin=True)

Username:  ········
Password:  ·······


Connection successful.


## 1. Add project

In [3]:
# Read projects
xl_path = r"../cl_vestland_stns.xlsx"
proj_df = pd.read_excel(xl_path, sheet_name="project")
assert len(proj_df) == 1

proj_df

,project_code,project_name,aquamonitor_id,contact,description
0,190246,CL Vestland,NaN,KAU,Critical loads calculations for Vestland


In [ ]:
## Add project
#proj_df.to_sql(
#    "projects", schema="niva", con=eng, if_exists="append", index=False,
#)

## 2. Add stations

In [4]:
# Read stations
xl_path = r"../cl_vestland_stns.xlsx"
stn_df = pd.read_excel(xl_path, sheet_name="stations")
stn_df["longitude"] = stn_df["longitude"].round(6)
stn_df["latitude"] = stn_df["latitude"].round(6)

stn_df.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,fpath_or_id
0,Sam_Tyss,Samnanger_Tysseelva,NaN,5.758580,60.374899,/home/jovyan/projects/critical_loads_2/shapefi...
1,Sam_Stor,Samnanger_Storelva,NaN,5.798443,60.384011,/home/jovyan/projects/critical_loads_2/shapefi...
2,Sam_Froe,Samnanger_Frolandselva,NaN,5.798510,60.380058,/home/jovyan/projects/critical_loads_2/shapefi...
3,Sam_Frok,Samnanger_Frolandskanalen,NaN,5.793918,60.381434,/home/jovyan/projects/critical_loads_2/shapefi...


In [5]:
# Build geom
stn_gdf = gpd.GeoDataFrame(
    stn_df,
    crs={"init": "epsg:4326"},
    geometry=gpd.points_from_xy(stn_df.longitude, stn_df.latitude),
).copy()
stn_gdf["geom"] = stn_gdf["geometry"].apply(lambda x: WKTElement(x.wkt, srid=4326))
del stn_gdf["fpath_or_id"], stn_gdf["geometry"]

stn_gdf.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,Sam_Tyss,Samnanger_Tysseelva,NaN,5.758580,60.374899,POINT (5.75858 60.374899)
1,Sam_Stor,Samnanger_Storelva,NaN,5.798443,60.384011,POINT (5.798443 60.384011)
2,Sam_Froe,Samnanger_Frolandselva,NaN,5.798510,60.380058,POINT (5.79851 60.380058)
3,Sam_Frok,Samnanger_Frolandskanalen,NaN,5.793918,60.381434,POINT (5.793918 60.381434)


In [ ]:
## Add stations
#stn_gdf.to_sql(
#    "stations",
#    schema="niva",
#    con=eng,
#    if_exists="append",
#    index=False,
#    dtype={"geom": Geometry("POINT", srid=4326)},
#    method="multi",
#    chunksize=1000,
#)

## 3. Add project-stations

In [6]:
# Build table for projects-stations
# Get station IDs
stn_codes = tuple(stn_df["station_code"].unique())
sql = text("SELECT station_id FROM niva.stations " "WHERE station_code IN :stn_codes")
prst_df = pd.read_sql(sql, params={"stn_codes": stn_codes}, con=eng)

# Get project ID
proj_code = proj_df["project_code"].values[0]
sql = text("SELECT project_id FROM niva.projects " "WHERE project_code = :proj_code")
proj_id = pd.read_sql(sql, params={"proj_code": str(proj_code)}, con=eng)[
    "project_id"
].iloc[0]

prst_df["project_id"] = proj_id
prst_df.head()

,station_id,project_id
0,78,2
1,79,2
2,80,2
3,81,2


In [ ]:
## Add projects-stations
#prst_df.to_sql(
#    "projects_stations",
#    schema="niva",
#    con=eng,
#    if_exists="append",
#    index=False,
#    method="multi",
#    chunksize=1000,
#)

## 4. Add catchments

In [7]:
# Loop over catchments
gdf_list = []
for idx, row in stn_df.iterrows():
    stn_code = row["station_code"]
    shp_path = row["fpath_or_id"]
    cat_gdf = gpd.read_file(shp_path)
    cat_gdf["dissolve"] = 1
    cat_gdf["station_code"] = stn_code
    cat_gdf = cat_gdf.dissolve(by="dissolve").reset_index()
    cat_gdf = cat_gdf[["station_code", "geometry"]]
    gdf_list.append(cat_gdf)

cat_gdf = pd.concat(gdf_list).reset_index(drop=True)
cat_gdf.head()

,station_code,geometry
0,Sam_Tyss,"POLYGON Z ((-6481.562 6728412.895 0.000, -6486..."
1,Sam_Stor,"POLYGON Z ((-4989.585 6733365.944 0.000, -5033..."
2,Sam_Froe,"POLYGON Z ((2525.877 6733985.895 0.000, 2583.5..."
3,Sam_Frok,"POLYGON Z ((2525.877 6733985.895 0.000, 2502.7..."


In [8]:
# Get station IDs from db
proj_id = 2
sql = text(
    "SELECT station_id, station_code FROM niva.stations "
    "WHERE station_id IN ( "
    "  SELECT station_id from niva.projects_stations "
    "  WHERE project_id = :proj_id)"
)
stn_ids = pd.read_sql(sql, params={"proj_id": proj_id}, con=eng)

# Join catchments
cat_gdf = cat_gdf.merge(stn_ids, on="station_code")

# Reproject to WGS84 GCS
cat_gdf = cat_gdf.to_crs({"init": "epsg:4326"})

# Cast to multi
cat_gdf["geometry"] = [
    MultiPolygon([feature]) if type(feature) == Polygon else feature
    for feature in cat_gdf["geometry"]
]

# Convert 3D to 2D
cat_gdf["geom"] = cat_gdf["geometry"].apply(
    lambda x: shapely.wkb.loads(shapely.wkb.dumps(x, output_dimension=2))
)

# Tidy
cat_gdf["geom"] = cat_gdf["geom"].apply(lambda x: WKTElement(x.wkt, srid=4326))
cat_gdf = cat_gdf[["station_id", "geom"]]

cat_gdf.head()

/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,station_id,geom
0,78,MULTIPOLYGON (((5.795751696587917 60.372852727...
1,79,MULTIPOLYGON (((5.809986354734426 60.418614454...
2,80,MULTIPOLYGON (((5.943204426129982 60.433416014...
3,81,MULTIPOLYGON (((5.943204426129982 60.433416014...


In [9]:
## Add catchments
#cat_gdf.to_sql(
#    "catchments",
#    schema="niva",
#    con=eng,
#    if_exists="append",
#    index=False,
#    dtype={"geom": Geometry("MULTIPOLYGON", srid=4326)},
#    method="multi",
#    chunksize=1000,
#)